In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets

import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
vocab_size = 10000
embedding_dim = 16


In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTCRJYO/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTCRJYO/imdb_reviews-test.tfrecord


In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
  
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
from collections import Counter
token_counts = Counter()

for col in training_sentences:
  token_counts.update(col.split())

In [ ]:
print("Total unique tokens :", len(token_counts))
print('\n'.join(map(str, token_counts.most_common(n=5))))
print('...')
print('\n'.join(map(str, token_counts.most_common()[-3:])))

In [ ]:
_ = plt.hist(list(token_counts.values()), range=[0, 10**4], bins=50, log=True)
plt.xlabel("Counts")

In [ ]:
min_count = 20

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = [token for token, count in token_counts.items() if count >= min_count]

# Add a special tokens for unknown and empty words
UNK, PAD = "UNK", "PAD"
tokens = [PAD, UNK] + tokens

In [ ]:
print("Tokens left:", len(tokens))

In [ ]:
token_to_id = {token: num for num, token in enumerate(tokens)}

In [ ]:
UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

def as_matrix(sequences, max_len=120):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = list(map(str.split, sequences))
        # sequences = sequences.split(' ')
    # max_len = min(max(map(len, sequences)), max_len or float('inf'))

    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))
    for i,seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

In [ ]:
len(training_sentences[0].split(' '))

116

In [ ]:
print(training_sentences[:1])
print("Matrix:")
print(as_matrix(training_sentences[:2]))

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."']
Matrix:
2
['b"This', 'was', 'an', 'absolutely', 'terrible', 'movie.', "Don't", 'be', 'lured', 'in', 'by', 'Christopher', 'Walken', 'or', 'Michael', 'Ironside.', 'Both', 'are', 'great', 'actors,', 'but', 'this', 'must', 'simply', 'be', 'their', 'worst', 'role', 'in', 'history.', 

In [ ]:
PAD_IX

0

In [ ]:
as_matrix(training_sentences[0]).shape

120


(116, 120)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
model = nn.Sequential(
          nn.Embedding(16755, embedding_dim, padding_idx=PAD_IX),
          Flatten(),

          nn.Linear(1920, 10),
          nn.ReLU(),

          nn.Linear(10, 2),
          nn.Sigmoid()
)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 10
compute_loss = nn.BCELoss()
batch_size = 1

In [ ]:
for epoch_i in range(num_epochs):
    for indices in range(0, len(training_sentences), batch_size):
        x_batch = torch.tensor(as_matrix(training_sentences[indices]), dtype=torch.int64)
        y_batch = torch.tensor(training_labels[idx], dtype=torch.int64)
        y_pred = model(x_batch)
        break
    break

In [ ]:
y_pred = model(x_batch)

In [ ]:
y_pred

tensor([[0.5000, 0.4470]], grad_fn=<SigmoidBackward>)